# Документация яндекс спеллера

>Похоже, можно использовать и в коммерческих целях, но лимит использования они не оговаривают

https://yandex.ru/legal/speller_api/index.html

In [1]:
import wandb
from src.datasets import load_datasets

orpho_dataset, punct_dataset = load_datasets()

c:\lang_check\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since ai-forever/spellcheck_benchmark couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'RUSpellRU' at C:\Users\Андрей Т\.cache\huggingface\datasets\ai-forever___spellcheck_benchmark\RUSpellRU\0.0.1\3395aa540689e4393c3e18d063e73a5b99d7f047 (last modified on Mon Jun 17 00:55:50 2024).


In [8]:
import requests
from typing import List, Tuple, Dict
from src.base import BaseModel


class YandexSpellerModel(BaseModel):

    def __init__(self):
        super().__init__()
        self.spell_service_url = (
            "https://speller.yandex.net/services/spellservice.json/checkText"
        )

    def _send_request(self, text: str) -> List[Dict]:
        """
        Отправляет запрос к Yandex Speller API для проверки текста.

        Args:
            text (str): Текст для проверки.

        Returns:
            List[Dict]: Список ошибок, найденных в тексте.
        """
        params = {"text": text}
        response = requests.get(self.spell_service_url, params=params)
        response.raise_for_status()
        return response.json()

    def _process_errors(self, text: str, errors: List[Dict]) -> Tuple[List[Dict], str]:
        """
        Обрабатывает ошибки и вносит исправления в текст.

        Args:
            text (str): Исходный текст.
            errors (List[Dict]): Список ошибок, найденных в тексте.

        Returns:
            Tuple[List[Dict], str]: Список словарей с информацией об ошибках и исправленный текст.
        """
        corrected_text = text
        corrections = []

        for error in errors:
            start_pos = error["pos"]
            end_pos = start_pos + error["len"]
            error_text = text[start_pos:end_pos]
            suggestions = error.get("s", [])

            if suggestions:
                corrected_text = (
                    corrected_text[:start_pos]
                    + suggestions[0]
                    + corrected_text[end_pos:]
                )

            correction = {
                "index": start_pos,
                "error": error_text,
                "suggestions": suggestions,
                "message": error.get("message", ""),
            }
            corrections.append(correction)

        return corrections, corrected_text

    def predict_verbose(self, text: str) -> Tuple[List[Dict], str]:
        """
        Проверяет текст на орфографические ошибки с использованием API Yandex Speller.

        Args:
            text (str): Текст для проверки.

        Returns:
            Tuple[List[Dict], str]: Список словарей с информацией об ошибках и исправленный текст.
        """
        errors = self._send_request(text)
        return self._process_errors(text, errors)

    def predict(self, text: str) -> str:
        """
        Возвращает текст с исправленными ошибками.

        Args:
            text (str): Текст для проверки.

        Returns:
            str: Исправленный текст.
        """
        _, corrected_text = self.predict_verbose(text)
        return corrected_text

    def evaluate(self, text: str, answer: str) -> Dict:
        """
        Оценка исправленного текста относительно ответа.
        Метод не реализован и должен быть переопределен в подклассах.

        Args:
            text (str): Исходный текст.
            answer (str): Ожидаемый текст.

        Returns:
            Dict: Оценка результатов.
        """
        raise NotImplementedError("This method should be overridden in subclasses")
        return super().evaluate(text, answer)

In [ ]:
model = YandexSpellerModel()
model.predict_verbose(
    "а так хочеться что-то мочь менять в этом мире не обезательно менять но обязательно быть способным это зделать"
)

Скоринг орфографии

In [4]:
from src.model_scorers import WandbSageModelScorer

sms = WandbSageModelScorer(dataset=orpho_dataset["test"], project="yandex_speller")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: andtun (obuchii). Use `wandb login --relogin` to force relogin


In [5]:
scoring_final_result, explanation = sms.score_explain(
    model, metrics=["errant", "ruspelleval"]
)
scoring_final_result

Calculating words metric: 100%|██████████| 2008/2008 [00:16<00:00, 123.22it/s]


{'SPELL_Precision': 61.97,
 'SPELL_Recall': 49.78,
 'SPELL_F1': 55.21,
 'YO_Precision': 0.0,
 'YO_Recall': 100.0,
 'YO_F1': 0.0,
 'PUNCT_Precision': 100.0,
 'PUNCT_Recall': 0.0,
 'PUNCT_F1': 0.0,
 'CASE_Precision': 100.0,
 'CASE_Recall': 100.0,
 'CASE_F1': 100.0,
 'Precision': 70.12,
 'Recall': 49.95,
 'F1': 58.34}

In [6]:
explanation

,Source,Truth,Model_result,Model_is_correct
0,﻿есть у вас оформленый и подписаный мною заказ,﻿есть у вас оформленный и подписанный мною заказ,﻿есть у вас оформленый и подписаный мною заказ,False
1,вот в инете откапал такую интеерсную статейку ...,вот в инете откопал такую интересную статейку ...,вот в инете откопал такую интересную статейку ...,True
2,я на всю жизнь запомню свое первое купание в з...,я на всю жизнь запомню свое первое купание в з...,я на всю жизнь запомню свое первое купание в з...,True
3,думаем что не ошибемся если скажем что выставк...,думаем что не ошибемся если скажем что выставк...,думаем что не ошибемся если скажем что выставк...,True
4,судьба человека может складываться очень разно...,судьба человека может складываться очень разно...,судьба человека может складываться очень разно...,True
...,...,...,...,...
2003,спасибо вам огромное за нормальную новость о е...,спасибо вам огромное за нормальную новость о е...,спасибо вам огромное за нормальную новость о е...,True
2004,более захватывающее и наглядное обучение возмо...,более захватывающее и наглядное обучение возмо...,более захватывающее и наглядное обучение возмо...,True
2005,и вобщем-то все понятно на фоне слухов застави...,и в общем-то все понятно на фоне слухов застав...,и в общем-то все понятно на фоне слухов застав...,False
2006,но всему есть придел,но всему есть предел,но всему есть предел,True


Скоринг пунктуации

In [7]:
sms.dataset = punct_dataset["test"]
sms.run_suffix = "_punct"

In [8]:
scoring_final_result, explanation = sms.score_explain(
    model, metrics=["errant", "ruspelleval"]
)
scoring_final_result

  0%|          | 0/2008 [00:00<?, ?it/s]

Calculating words metric: 100%|██████████| 2008/2008 [00:10<00:00, 198.91it/s]


{'SPELL_Precision': 58.2,
 'SPELL_Recall': 36.33,
 'SPELL_F1': 44.73,
 'CASE_Precision': 100.0,
 'CASE_Recall': 0.0,
 'CASE_F1': 0.0,
 'PUNCT_Precision': 100.0,
 'PUNCT_Recall': 0.0,
 'PUNCT_F1': 0.0,
 'YO_Precision': 50.0,
 'YO_Recall': 0.83,
 'YO_F1': 1.63,
 'Precision': 70.97,
 'Recall': 48.19,
 'F1': 57.41}

In [9]:
explanation

,Source,Truth,Model_result,Model_is_correct
0,а так хочеться что-то мочь менять в этом мире ...,А так хочется что-то мочь менять в этом мире: ...,а так хочется что-то мочь менять в этом мире н...,False
1,давольно милый и летом и зимой обогреваемый те...,"Довольно милый, и летом, и зимой обогреваемый ...",довольно милый и летом и зимой обогреваемый те...,False
2,бывают такие моменты когда хочеться зделать чт...,"Бывают такие моменты, когда хочется сделать чт...",бывают такие моменты когда хочется зсделатьчто...,False
3,﻿есть у вас оформленый и подписаный мною заказ,﻿Есть у вас оформленный и подписанный мною заказ?,﻿есть у вас оформленый и подписаный мною заказ,False
4,вот в инете откапал такую интеерсную статейку ...,"Вот в инете откопал такую интересную статейку,...",вот в инете откопал такую интересную статейку ...,False
...,...,...,...,...
2003,спасибо вам огромное за нормальную новость о е...,Спасибо вам огромное за нормальную новость о Ё...,спасибо вам огромное за нормальную новость о е...,False
2004,более захватывающее и наглядное обучение возмо...,Более захватывающее и наглядное обучение возмо...,более захватывающее и наглядное обучение возмо...,False
2005,и вобщем-то все понятно на фоне слухов застави...,"И, в общем-то, всё понятно на фоне слухов: зас...",и в общем-то все понятно на фоне слухов застав...,False
2006,но всему есть придел,Но всему есть предел.,но всему есть предел,False


In [10]:
wandb.run.finish()

# Выводы

https://wandb.ai/obuchii/NLP%20Scoring/runs/mjvb28xc

|  Задача | Errant Precision | Errant Recall | Errant F1 | Precision | Recall | F1
|----------|----------|----------|----------|----------|----------|----------|
| Spelling   | 61.97   | 49.78   | 55.21   | 70.12   | 49.95   | 58.34   |

-------


|  Задача | Punct Precision | Punct Recall | Punct F1 | Case Precision | Case Recall | Case F1
|----------|----------|----------|----------|----------|----------|----------|
| Punctuation   | 100   | 0   | 0    | 100.0  | 0   | 0    |


Модель плохенько справляется со спеллингом, обычно на сложных примерах.

Модель не умеет исправлять пунктуационные ошибки.

Не исправляет пунктуацию, работает супер долго, есть риск vendor lock

Модель использовать не будем
